#Packages

In [ ]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install wikipedia
!pip install serp
!pip install google-search-results
!pip install arxiv
!pip install langchain_experimental
!pip install langchain_community --quiet

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "Your own OPEANAI_API_KEY"
os.environ['COHERE_API_KEY'] = "Your own COHERE_API_KEY"

#Better way
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
#os.environ['COHERE_API_KEY'] = userdata.get("COHERE_API_KEY")

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI, Cohere
from langchain.chat_models import ChatOpenAI

# Agent Example

In [ ]:
llm=OpenAI(temperature=0)
tools=load_tools(["wikipedia"],llm=llm)
agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

question="""
"What is the law that is related to UPI cyber crimes in India?
How many years back it was introduced?
What is the name of the Law and the Section?
"""
agent.run(question)

In [ ]:
llm=ChatOpenAI(temperature=0)
tools=load_tools(["wikipedia"],llm=llm)
agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

agent.run("When was MS-Excel first released? How many years back?")

In [ ]:
# Calculator tool
llm=ChatOpenAI(temperature=0)
tools=load_tools(["llm-math"],llm=llm)
agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

agent.run("what is the cube root of 999")

# Tools

In [ ]:
os.environ["SERPAPI_API_KEY"]="Your Key"
os.environ['SERPAPI_API_KEY'] = userdata.get("SERPAPI_API_KEY")

In [ ]:
!pip install serp
!pip install google-search-results

In [ ]:
llm=ChatOpenAI(temperature=0)

tools=load_tools(["serpapi", "llm-math"],llm=llm)

agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

agent.run("When was MS-Excel first released? How many years back?")

# PPT Maker App

In [ ]:
!pip install arxiv

In [ ]:
from langchain.prompts import PromptTemplate

llm=OpenAI(temperature=0.3)

tools=load_tools(["serpapi","arxiv","wikipedia"],llm=llm)

agent=initialize_agent(tools,
                       llm,
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)

template="""
take the topic name as input. Topic Name : {Topic_name}
Extract the information from all the tools. Generate atleast an essay of 1000 words
Create a powerpoint friendly content from the essay
Give the output in 10 sections. each section title and minimum 6 bullet points
"""
prompt_template=PromptTemplate(template=template, input_variables=["Topic_name"])


agent.run(prompt_template.format(Topic_name="Investing in gold"))

## Output in markdown

Investing in gold can be a lucrative investment strategy, but it is important to understand the market trends and potential risks involved. Gold has been a popular investment choice for centuries, and its value has remained relatively stable compared to other assets. However, it is subject to speculation and volatility like any other market. In this presentation, we will discuss the basics of investing in gold, its historical significance, and current market trends to help you make informed decisions when considering this investment option.

Slide 1: Introduction
- Definition of gold as an investment
- Why invest in gold?
- Overview of presentation

Slide 2: Historical Significance of Gold
- Gold as a currency and store of value throughout history
- The gold standard and its impact on the global economy
- Gold's role in times of crisis and uncertainty

Slide 3: Types of Gold Investments
- Physical gold (coins, bars, jewelry)
- Gold ETFs and mutual funds
- Gold mining stocks
- Gold futures and options

Slide 4: Pros of Investing in Gold
- Diversification of portfolio
- Hedge against inflation and currency

# CSV Agent

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents import AgentType

In [ ]:
!wget "https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Bank%20Tele%20Marketing/bank_market.csv"

In [ ]:
llm=OpenAI(temperature=0)
agent=create_csv_agent(llm=llm,
                       path="bank_market.csv",
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       allow_dangerous_code=True,#Throws an error without this
                       verbose=True)
result=agent.run("What are the column names in the dataset?")
print(result)

In [ ]:
print(agent.run("What is the average age of the customers?"))

In [ ]:
print(agent.run("Are there any outliers in the dataset?"))

## Working with Multiple CSV files

In [ ]:
!wget  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/TV%20Commercial%20Slots%20Analysis/orders.csv
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/TV%20Commercial%20Slots%20Analysis/slots.csv

In [ ]:
agent = create_csv_agent(llm=llm,
    path=["orders.csv","slots.csv"],
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,#Throws an error without this
    verbose=True
    )

In [ ]:
agent.run("What are the columns that are common in both the datasets?")

In [ ]:
agent.run("If I do an inner join based on Unique_id, How many records will be there in the resultant dataset?")

In [ ]:
agent.run("If I do an Outer join based on Unique_id', How many records will be there in the resultant dataset?")

#App - Talk to your Data

In [ ]:
%%writefile requirements.txt
langchain
langchain-openai
langchain-experimental
streamlit
openai
langchain_community

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
%%writefile app.py
import streamlit as st
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents import AgentType
from langchain.utilities import PythonREPL

def main():
  st.set_page_config(page_title="AI Assistant ", page_icon=":robot:")
  st.title("AI Assistant :wave:")

  with st.sidebar:
    st.sidebar.title("Your File :page_with_curl:")
    input_csv=st.file_uploader("Upload a CSV file here", type=["csv"])

  if input_csv:
    user_question=st.text_input("Ask a question on your data")
    if user_question is not None:
      agent=create_csv_agent(OpenAI(temperature=0),
                            input_csv,
                            allow_dangerous_code=True,
                            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                            verbose=True)
      result=agent.run(user_question)
      st.write(result)



if __name__ == "__main__":
  main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

# Working with Excel files

In [ ]:
!wget  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/PowerBI_Sessions_Data/healthcare_dataset_stroke_data_v1.xlsx

In [ ]:
import pandas as pd
excel_data_frame=pd.read_excel("healthcare_dataset_stroke_data_v1.xlsx", sheet_name=1)
excel_data_frame.to_csv("healthcare_dataset_stroke_data_v1.csv")

In [ ]:
llm=OpenAI(temperature=0)
agent = create_csv_agent(
    llm=llm,
    path="healthcare_dataset_stroke_data_v1.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    )

In [ ]:
agent.run("Print top5 rows")

In [ ]:
agent.run("What are the column names")

In [ ]:
agent.run("Convert the first row as column names and print the column names")

In [ ]:
agent.run("Treat the first row as column names. How many customers are male?")

# SQL Agent

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/chinook/chinook.db

#Dataset Description https://github.com/lerocha/chinook-database

In [ ]:
db = SQLDatabase.from_uri("sqlite:///chinook.db")

#For MySql Server

#db_user = "user"
#db_password = "password"
#db_host = "host"
#db_name = "db_name"
#db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)

In [ ]:
agent_executor.invoke("Describe the schema of the playlist table")

In [ ]:
agent_executor.run("How many rows are there in the employees table?")

In [ ]:
agent_executor.run("How many different artistis are there?")

## Problem with the Agent

In [ ]:
agent_executor.run("How many rows are there in playlisttrack table")

In [ ]:
agent_executor.run("Delete the table playlisttrack from the database and print the table names")

In [ ]:
agent_executor.run("How many rows are there in the playlisttrack table?")

## Prefix template

In [ ]:
prefix_template = """You are an agent designed to interact with a SQL database.
INSERT, UPDATE, DELETE, DROP etc. statements are not allowed.
CREATE TABLE statements are not allowed.
DROP TABLE statements are not allowed.
INSERT INTO statements are not allowed.
Never make any changes to the database.
Avoid any actions that could potentially compromise the integrity or security of the database.
Ensure that all interactions with the database are read-only and non-destructive.
If the question does not seem related to the database, just return I do not know as the answer.
"""

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

modified_agent = create_sql_agent(
    llm=ChatOpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=prefix_template,
)

In [ ]:
modified_agent.run("drop the table generes from the database")

In [ ]:
modified_agent.run("print all the names of the tables in the database")

## PromptTemplate

In [ ]:
from langchain import PromptTemplate

template="""
take the input query here : {input_query}
You are an agent designed to interact with a SQL database.
INSERT, UPDATE, DELETE, DROP etc. statements are not allowed.
CREATE TABLE statements are not allowed.
DROP TABLE statements are not allowed.
INSERT INTO statements are not allowed.
Never make any changes to the database.
Avoid any actions that could potentially compromise the integrity or security of the database.
Ensure that all interactions with the database are read-only and non-destructive.
If the question does not seem related to the database, just return I do not know as the answer.
"""
prompt_template=PromptTemplate(template=template, input_variables=["input_query"])

agent_executor.run(prompt_template.format(input_query="drop the table invoices from the database"))

In [ ]:
!pip install langchainhub

In [ ]:
from langchain import hub
base_prompt = hub.pull("langchain-ai/openai-functions-template")
print(base_prompt)

In [ ]:
instructions="""
You are an agent designed to interact with a SQL database.
INSERT, UPDATE, DELETE, DROP etc. statements are not allowed.
CREATE TABLE statements are not allowed.
DROP TABLE statements are not allowed.
INSERT INTO statements are not allowed.
Never make any changes to the database.
Avoid any actions that could potentially compromise the integrity or security of the database.
Ensure that all interactions with the database are read-only and non-destructive.
If the question does not seem related to the database, just return I do not know as the answer.
"""
prompt = base_prompt.partial(instructions=instructions)
print(prompt)


In [ ]:
agent_executor.run("Drop the table genres from the database ")

# Custom Tools

In [ ]:
from langchain.agents import tool

In [ ]:
llm=OpenAI(temperature=0)
tools=load_tools(["wikipedia"], llm=llm)
agent=initialize_agent(tools,
                       llm,
                       agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)
agent.run("What is today's date?")

## Defining your own tool

In [ ]:
@tool

def date_tool(text:str)->str:
  """
  This function takes the input as an empty string and returns the current date
  Only use this function for the current date and time do not use it for other tasks
  """
  import datetime
  return datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
llm=OpenAI(temperature=0)
tools=load_tools(["wikipedia"], llm=llm)
tools.append(date_tool)
agent=initialize_agent(tools,
                       llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                       verbose=True)
agent.run("What is today's date?")

# Pandas Dataframe Agent

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [ ]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Census%20Income%20Data/Income_data.csv")

In [ ]:
df_agent = create_pandas_dataframe_agent(
    llm=OpenAI(temperature=0),
    df=df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    )

In [ ]:
df_agent.invoke("how many rows and columns are there in the data?")

In [ ]:
df_agent.invoke("Print all the column names")

In [ ]:
df_agent.invoke("Create a frequency table for the marital-status column")

In [ ]:
df_agent.invoke("Create a bar chart for the occupation column")

# Pandas AI


In [ ]:
!pip install pandasai

In [ ]:
from pandasai import SmartDataframe
import pandas as pd

In [ ]:
#You can get your free API key signing up at https://pandabi.ai
import os
os.environ['PANDASAI_API_KEY'] = "Your Key"

In [ ]:
kc_house_price=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/kc_house_data/kc_house_data.csv")

In [ ]:
#A SmartDataframe inherits all the properties and methods from the pd.DataFrame, but also adds conversational features to it.
smart_kc_house_price=SmartDataframe(kc_house_price)

## Having a Conversation with the dataframe

In [ ]:
smart_kc_house_price.chat("How many rows and columns are there in the kc_house_price data?")

In [ ]:
smart_kc_house_price.chat("Print the Column names in the kc_house_price data. Give the output in tabular format")

In [ ]:
# To get idea on the background code
print(smart_kc_house_price.last_code_generated)

## Plotting Options

In [ ]:
smart_kc_house_price.chat("Plot barchart for the number of bedrooms column")

In [ ]:
smart_kc_house_price.chat("Plot barchart for the number of bedrooms column. Keep each bar with a different color")

In [ ]:
smart_kc_house_price.chat("Draw a box plot for the price column in the kc_house_price data.")

In [ ]:
prompt="""
Draw a geo map chat to show the location of the house and indicate zipcode  using colour
Use latitude and logitude values to draw the geo map chart on kc house price data
"""
smart_kc_house_price.chat(prompt)